In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from fastapi import FastAPI
from pydantic import BaseModel
from datetime import datetime
import uvicorn

In [7]:
class PredictionInput(BaseModel):
    date: str
    product_category: str
    region: str
    agent_id: str
    marketing_spend: float
    lead_count: int

In [8]:
# Analyzing the data
import pandas as pd

def check_data_issues(df):
    """Checks dataset for issues like missing values, invalid dates, datatypes, and duplicates."""
    print("Data Issues Report:")
    print(f"Total Rows: {len(df)}")
    
    # 1. Missing Values
    missing = df.isna().sum()
    print("\nMissing Values (isna()):")
    for col, count in missing.items():
        if count > 0:
            print(f"- {col}: {count} missing values")
    
    # 2. Invalid Categorical Values ('Null', 'N/A', empty)
    categorical_cols = ['product_category', 'region', 'agent_id']
    print("\nInvalid Categorical Values ('Null', 'N/A', empty):")
    for col in categorical_cols:
        if col in df.columns:
            invalid = df[col].isin(['Null', 'N/A', '']).sum()
            if invalid > 0:
                print(f"- {col}: {invalid} invalid values")
        else:
            print(f"- {col}: Column not found")
    
    # 3. Invalid Dates
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')
        invalid_dates = df['date'].isna().sum() + (df['date'].dt.year != 2025).sum()
        print(f"\nInvalid Dates (unparseable or not in 2025): {invalid_dates}")
    else:
        print("\nInvalid Dates: 'date' column not found")
    
    # 4. Non-Numeric Values in Numeric Columns
    numeric_cols = ['marketing_spend', 'lead_count', 'revenue']
    print("\nNon-Numeric Values in Numeric Columns:")
    for col in numeric_cols:
        if col in df.columns:
            non_numeric = pd.to_numeric(df[col], errors='coerce').isna().sum()
            if non_numeric > 0:
                print(f"- {col}: {non_numeric} non-numeric values")
            negative = (pd.to_numeric(df[col], errors='coerce') < 0).sum()
            if negative > 0:
                print(f"- {col}: {negative} negative values")
        else:
            print(f"- {col}: Column not found")
    
    # 5. Duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate Rows (duplicated()): {duplicates}")

if __name__ == "__main__":
    # Load CSV file directly
    df = pd.read_csv("/kaggle/input/forecasting-dataset/dummy_revenue_forecasting_data.csv")
    check_data_issues(df)

Data Issues Report:
Total Rows: 610998

Missing Values (isna()):
- agent_id: 22242 missing values
- marketing_spend: 1080 missing values
- lead_count: 1080 missing values
- revenue: 1080 missing values

Invalid Categorical Values ('Null', 'N/A', empty):
- region: 6602 invalid values
- agent_id: 2 invalid values

Invalid Dates (unparseable or not in 2025): 156890

Non-Numeric Values in Numeric Columns:
- marketing_spend: 12810 non-numeric values


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


- lead_count: 1080 non-numeric values
- revenue: 6562 non-numeric values

Duplicate Rows (duplicated()): 620


In [5]:
from sklearn.model_selection import train_test_split

X = df[features]
y = df['revenue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
from xgboost import XGBRegressor


In [8]:
# ✅ Weekly Revenue Forecasting Pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import joblib

# Load dataset
file_path = "/kaggle/input/forecasting-dataset/dummy_revenue_forecasting_data.csv"
df = pd.read_csv(file_path)

# Parse and filter date
df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)
df = df[df['date'].dt.year == 2025]

# Clean & convert columns
df.replace({'region': {'Null': np.nan, 'N/A': np.nan, '': np.nan},
            'agent_id': {'Null': np.nan, 'N/A': np.nan, '': np.nan}}, inplace=True)
for col in ['marketing_spend', 'lead_count', 'revenue']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['date', 'product_category', 'region', 'agent_id',
                  'marketing_spend', 'lead_count', 'revenue'], inplace=True)
df.drop_duplicates(inplace=True)

# 🔄 Weekly aggregation
df['week'] = df['date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_df = df.groupby(['week', 'agent_id', 'product_category', 'region']).agg({
    'marketing_spend': 'sum',
    'lead_count': 'sum',
    'revenue': 'sum'
}).reset_index()

# Sort
weekly_df.sort_values(by=['agent_id', 'week'], inplace=True)

# Lag features
for lag in [1, 2, 3]:
    weekly_df[f'revenue_lag_{lag}'] = weekly_df.groupby('agent_id')['revenue'].shift(lag)
    weekly_df[f'marketing_lag_{lag}'] = weekly_df.groupby('agent_id')['marketing_spend'].shift(lag)
    weekly_df[f'leads_lag_{lag}'] = weekly_df.groupby('agent_id')['lead_count'].shift(lag)

# Rolling means
weekly_df['revenue_roll_mean_3'] = weekly_df.groupby('agent_id')['revenue'].transform(lambda x: x.shift(1).rolling(3).mean())

# Temporal features
weekly_df['weekofyear'] = weekly_df['week'].dt.isocalendar().week
weekly_df['month'] = weekly_df['week'].dt.month

# Encoding
for col in ['product_category', 'region', 'agent_id']:
    weekly_df[f'{col}_freq'] = weekly_df[col].map(weekly_df[col].value_counts(normalize=True))
    weekly_df[f'{col}_mean_rev'] = weekly_df[col].map(weekly_df.groupby(col)['revenue'].mean())

# Drop NA from lag features
weekly_df.dropna(inplace=True)

# Log target
y = np.log1p(weekly_df['revenue'])
features = [
    'marketing_spend', 'lead_count',
    'revenue_lag_1', 'revenue_lag_2', 'revenue_lag_3', 'revenue_roll_mean_3',
    'marketing_lag_1', 'marketing_lag_2', 'marketing_lag_3',
    'leads_lag_1', 'leads_lag_2', 'leads_lag_3',
    'weekofyear', 'month',
    'product_category_freq', 'region_freq', 'agent_id_freq',
    'product_category_mean_rev', 'region_mean_rev', 'agent_id_mean_rev'
]
X = weekly_df[features]

# Split
tsplit = int(len(X) * 0.8)
X_train, X_test = X.iloc[:tsplit], X.iloc[tsplit:]
y_train, y_test = y.iloc[:tsplit], y.iloc[tsplit:]

# Models
cat = CatBoostRegressor(verbose=0, iterations=2000, learning_rate=0.03, depth=6, random_seed=42)
lgb = LGBMRegressor(n_estimators=2000, learning_rate=0.03, num_leaves=50, random_state=42)
xgb = XGBRegressor(n_estimators=2000, learning_rate=0.03, max_depth=6, tree_method='hist', subsample=0.8,
                   colsample_bytree=0.8, random_state=42)

# Stacking
stack = StackingRegressor(
    estimators=[('cat', cat), ('lgb', lgb), ('xgb', xgb)],
    final_estimator=LGBMRegressor(n_estimators=500, learning_rate=0.05, num_leaves=30, random_state=42),
    passthrough=True
)

# Fit
stack.fit(X_train, y_train)

# Predict
y_pred_log = stack.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Metrics
print(f"✅ MAE: {mean_absolute_error(y_true, y_pred):.2f}")
print(f"✅ R²: {r2_score(y_true, y_pred):.4f}")
print(f"✅ MAPE: {mean_absolute_percentage_error(y_true, y_pred):.4f}")

# Save
joblib.dump(stack, "/kaggle/working/final_model2.pkl")
joblib.dump(features, "/kaggle/working/final_features2.pkl")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3100
[LightGBM] [Info] Number of data points in the train set: 2216, number of used features: 17
[LightGBM] [Info] Start training from score 16.294248
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000459 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3099
[LightGBM] [Info] Number of data points in the train set: 1772, number of used features: 17
[LightGBM] [Info] Start training from score 16.295174
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3099
[LightGBM] [Info] Number of data points in the train set: 1773, number of used features: 17
[LightGBM] [Info] Start trai

['/kaggle/working/final_features2.pkl']

In [13]:
import joblib
import numpy as np
import pandas as pd

# Load the model and features
model = joblib.load("/kaggle/working/final_model2.pkl")
features = joblib.load("/kaggle/working/final_features2.pkl")

# ✅ Example input (you can modify values here)
example_input = {
   "marketing_spend": 45000,
    "lead_count": 350,
    "revenue_lag_1": 420000,
    "revenue_lag_2": 390000,
    "revenue_lag_3": 370000,
    "revenue_roll_mean_3": 393333.33,
    "marketing_lag_1": 40000,
    "marketing_lag_2": 38000,
    "marketing_lag_3": 36000,
    "leads_lag_1": 300,
    "leads_lag_2": 280,
    "leads_lag_3": 260,
    "weekofyear": 27,
    "month": 7,
    "product_category_freq": 0.12,
    "region_freq": 0.18,
    "agent_id_freq": 0.004,
    "product_category_mean_rev": 410000,
    "region_mean_rev": 395000,
    "agent_id_mean_rev": 385000
}

# Convert to DataFrame
X_new = pd.DataFrame([example_input])[features]

# Predict (model was trained on log1p)
log_pred = model.predict(X_new)
revenue_pred = np.expm1(log_pred)[0]

print(f"✅ Forecasted Weekly Revenue: ₹{revenue_pred:,.2f}")


✅ Forecasted Weekly Revenue: ₹4,092,376.67


In [12]:
df.head(20)

,date,product_category,region,agent_id,marketing_spend,lead_count,revenue,week
0,2025-03-29,Cosmetic,North,A4,21000.0,170.0,75000.0,2025-03-24
1,2025-03-29,Logistic,South,A5,14000.0,60.0,43000.0,2025-03-24
2,2025-03-29,Fintech,West,A5,31000.0,70.0,89000.0,2025-03-24
3,2025-03-30,Cosmetic,North,A3,47000.0,70.0,86000.0,2025-03-24
4,2025-03-30,FMCG,West,A9,19000.0,80.0,74000.0,2025-03-24
5,2025-03-30,FMCG,East,A8,5000.0,120.0,86000.0,2025-03-24
6,2025-03-31,FMCG,East,A3,5000.0,170.0,70000.0,2025-03-31
7,2025-03-31,Insurance,South,A3,20000.0,140.0,72000.0,2025-03-31
8,2025-03-31,Fintech,South,A9,34000.0,190.0,51000.0,2025-03-31
9,2025-04-01,Insurance,East,A5,22000.0,180.0,71000.0,2025-03-31
